# Table of Contents
 <p><div class="lev1"><a href="#How-to-build-a-2D-slider-crank-system-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>How to build a 2D slider-crank system</a></div>

# How to build a 2D slider-crank system

F. Dubois - 2016

In [ ]:
#Initialization

import os
# For windows users
#import sys
#sys.path.append('C:\gshdhgfhsgf\\build')

import math, numpy
from pylmgc90 import pre

if not os.path.isdir('DATBOX'):
  os.mkdir('DATBOX')

# 2D
dim = 2

# containers
#   * bodies
bodies = pre.avatars()
#   * materials
mats   = pre.materials()
#   * see tables
svs    = pre.see_tables()
#   * contact laws
tacts  = pre.tact_behavs()

tt=0.


Defining a material/model suitable for rigid objects. 

In [ ]:
mat = pre.material(name='TDURx', materialType='RIGID', density=1000.)
mats+=mat

mod = pre.model(name='rigid', physics='MECAx', element='Rxx2D', dimension=dim)

Crank:
 motor ...

In [ ]:
disk = pre.avatar(dimension=2)
# new node
no = pre.node(coor=numpy.array([0.,0.]),number=1)
# adding no to avatar
disk.addNode( no )
# new element 
disk.addBulk( pre.rigid2d() )
#
disk.defineGroups()
#
disk.defineModel(model=mod)
disk.defineMaterial(material=mat)
#
disk.addContactors(shape='DISKx', color='BLUEx', byrd=0.5)
disk.addContactors(shape='PT2Dx', color='VERTx', shift=[0., 0.])
disk.addContactors(shape='PT2Dx', color='VERTx', shift=[0.1,0.])
disk.computeRigidProperties()

disk.imposeDrivenDof(component=[1, 2], dofty='vlocy')
disk.imposeDrivenDof(component=[3], dofty='vlocy',ct=1.,rampi=1.)

bodies+=disk

 ... arm 1 ...

In [ ]:
arm1 = pre.avatar(dimension=2)
# new node
no = pre.node(coor=numpy.array([1,0.]),number=1)
# adding no to avatar
arm1.addNode( no )
# new element 
arm1.addBulk( pre.rigid2d() )
#
arm1.defineGroups()
#
arm1.defineModel(model=mod)
arm1.defineMaterial(material=mat)
#
arm1.addContactors(shape='JONCx', color='WALLx', axe1=1., axe2=0.1)
arm1.addContactors(shape='PT2Dx', color='VERTx', shift=[-1.,0.])
arm1.addContactors(shape='PT2Dx', color='VERTx', shift=[-0.9,0.])
arm1.addContactors(shape='PT2Dx', color='VERTx', shift=[1.,0.])
arm1.computeRigidProperties()
bodies+=arm1

 ... arm2 ...

In [ ]:
arm2 = pre.avatar(dimension=2)
# new node
no = pre.node(coor=numpy.array([5.,0.]),number=1)
# adding no to avatar
arm2.addNode( no )
# new element 
arm2.addBulk( pre.rigid2d() )
#
arm2.defineGroups()
#
arm2.defineModel(model=mod)
arm2.defineMaterial(material=mat)
#
arm2.addContactors(shape='JONCx', color='WALLx', axe1=3., axe2=0.1)
arm2.addContactors(shape='PT2Dx', color='VERTx', shift=[-3.,0.])
arm2.addContactors(shape='PT2Dx', color='VERTx', shift=[3.,0.])
arm2.computeRigidProperties()

bodies+=arm2

slider

In [ ]:
slider = pre.avatar(dimension=2)
# new node
no = pre.node(coor=numpy.array([8.,0.]),number=1)
# adding no to avatar
slider.addNode( no )
# new element 
slider.addBulk( pre.rigid2d() )
#
slider.defineGroups()
#
slider.defineModel(model=mod)
slider.defineMaterial(material=mat)
#
slider.addContactors(shape='POLYG', color='REDxx', nb_vertices=4, 
                     vertices=numpy.array([[-1.,-0.5],[1.,-0.5],[1.,0.5],[-1.,0.5]]))
slider.addContactors(shape='PT2Dx', color='VERTx', shift=[0.,0.])

slider.computeRigidProperties()

bodies+=slider

up = pre.rigidJonc(axe1=5., axe2=0.1, center=[8.,0.61], model=mod, material=mat, color='REDxx')
up.imposeDrivenDof(component=[1, 2, 3], dofty='vlocy')
bodies+=up

down = pre.rigidJonc(axe1=5., axe2=0.1, center=[8.,-0.61], model=mod, material=mat, color='REDxx')
down.imposeDrivenDof(component=[1, 2, 3], dofty='vlocy')
bodies+=down 

contact laws

In [ ]:
lcpl  = pre.tact_behav(name='xxxxx', law='COUPLED_DOF')
tacts+= lcpl
liqs  = pre.tact_behav(name='iqsc0', law='IQS_CLB', fric=0.3)
tacts+= liqs

sv1 = pre.see_table(CorpsCandidat   ='RBDY2', candidat   ='PT2Dx', colorCandidat   ='VERTx',
                    behav=lcpl, 
                    CorpsAntagoniste='RBDY2', antagoniste='PT2Dx', colorAntagoniste='VERTx',
                    alert=0.05)
svs+=sv1

sv2 = pre.see_table(CorpsCandidat   ='RBDY2', candidat   ='POLYG', colorCandidat   ='REDxx',
                    behav=liqs, 
                    CorpsAntagoniste='RBDY2', antagoniste='JONCx', colorAntagoniste='REDxx',
                    alert=0.05)
svs+=sv2


In [ ]:
# writing files
pre.writeBodies(bodies,chemin='DATBOX')
pre.writeBulkBehav(mats,chemin='DATBOX',dim=dim,gravy=[0.,0.,0.])
pre.writeTactBehav(tacts,svs,chemin='DATBOX')
pre.writeDrvDof(bodies,chemin='DATBOX')
pre.writeDofIni(bodies,chemin='DATBOX')
pre.writeVlocRlocIni(chemin='DATBOX')

post = pre.postpro_commands()
my_command = pre.postpro_command(name='SOLVER INFORMATIONS', step=1)
post.addCommand(my_command)
#
#floor_dte=pre.postpro_command(type='DOUBLET TORQUE EVOLUTION', step=1,
#                              doublets=[(arm1,arm2)])
#post.addCommand(floor_dte)
#
#
pre.writePostpro(commands=post, parts=bodies, path='DATBOX/')

try:
  pre.visuAvatars(bodies)
except:
  pass

In [ ]:

#  disk.addContactors(shape='POLYG', color='REDxx', nb_vertices=4, 
#                     vertices=np.array([[-1.,-1.],[1.,-1.],[1.,1.],[-1.,1.]]))
